# Collaborative filtering - Matrix Factorization with Surprise

In this notebook, you will learn another collaborative filtering technique called Matrix Factorization, more specifically we will use `Singular Value Decomposition (SVD)` to factorize the **user-item** interaction matrix. 
The **benefit** of using matrix factorization is that it can **capture** the **latent features** underlying the interactions between users and items. This allows us to make recommendations based on the user-item interaction matrix even if we have missing values.

We will use again  the Scikit-Surprise library to build a **SVD model** and make recommendations.

first we import the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
from copy import copy
from surprise import Dataset
from surprise import Reader
from surprise import KNNWithMeans, SVD
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from surprise import accuracy
from collections import defaultdict

Let's load our rating data. It contains the necessary `user_id`, `item_id` and the `rating` users gave to the fish items. Additionally it has some nice-to-have information about the fish items. There are 500 users with 300 rated fishes each. 

In [ ]:
# Loading the dataset from the csv file
df = pd.read_csv('data/user_item_ratings.csv')
df.head(3)

The Scikit-Surprise library we want to use does not work with pandas DataFrames but with Dataset objects. So we need to create a Dataset object from our DataFrame. We also need to define the possible ratings with the Reader class.

In [ ]:
# defines possible ratings
reader = Reader(rating_scale=(1, 10))
# Loads Pandas dataframe
data = Dataset.load_from_df(df[["user_id", "item_id", "rating"]], reader)

In order to validate our models we need to split our data into a trainset, which we will use to train our models. And a testset to validate the ability of our models to predict on unseen data. 

In [ ]:
# Splitting the data into training and test set
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

Let's start with the modeling!

## Singular Value Decomposition

Another way to estimate the user ratings is **Singular Value Decomposition (SVD)**. **Singular value decomposition** is a matrix decomposition, where we decompose a given **m x n** matrix $A$ into three matrices **$U$**, $\Sigma$ and $V$ with the dimensions **m x r**, **r x r** and **n x r** respectively. Here r is usually small compared to m and n. And $\Sigma$ is non-zero only on the diagonal.

![](./images/SVD_USigmaV.png) 

By decomposing the matrix $A$ in this way, we drastically reduce the number of elements to be stored. Let $m = n = 100$ and $r = 5$ then the original matrix $A$ has $100 \times 100 = 10000$ elements. Whereas $U$ has $100 \times 5 = 500$, $\Sigma$ has $5$ (only diagonal elements) and $V$ again has $100 \times 5 = 500$ elements leading to a total of only $1005$ elements as opposed to the $10000$ elements of the original matrix $A$.

The rows of m of our user-item-rating matrix $A$ refer to the users and the n columns to the items (fishes in our case). The introduced **latent factors** (r in above example) can be interpreted as characteristics of the users for $U$ and item characteristics for $V$. So the latent factors are features of items or users, generated by our **SVD algorithm**. In our case they can tell us how much a user likes a certain `visual_effect` or how much a fish shows said `visual_effect`. $\Sigma$ then models the importance of each `visual_effect`.

The **actual implementation** of the algorithm in the **surprise library** will be outlined here to show an example of how to find an optimal decomposition. It became popular due to Simon Funk and his contribution to the Netflix competition. 

Here the rating that user u gives item i is denoted by $r_{ui}$ and will be estimated by:


$$
\hat r_{ui} = \mu + b_u + b_i + q^T_i p_u
$$

where 

- (u,i): **user-item pair**
- $\mu$: **average rating of all items**
- $b_i$: **average rating of item i minus $\mu$**
- $b_u$: **average rating given by user u minus $\mu$**
- $q_i$: **latent item factor vector**
- $p_u$: **latent user factor vector**
- $q^T_i p_u$: **dot product of the latent factors**

As shown in the formula above, the matrix decomposition is done by using only two matrices **$Q$** and **$P$**. The matrix **$Q$** contains the latent factors of the items and the matrix **$P$** contains the latent factors of the users. The latent factors and the biases are learned by minimizing the regularized squared error:


$$
\sum_{r_{ui} \in R_{train}}(r_{ui} - \hat r_{ui})^2 + \lambda(b_i^2 + b_u^2 + ||q_i||^2 + ||p_u||^2)
$$

The minimization is performed by a simple stochastic gradient descent (parameters are updated iteratively):

$$
\begin{align*}
b_u &\leftarrow b_u + \gamma(e_{ui} - \lambda b_u)\\
b_i &\leftarrow b_i + \gamma(e_{ui} - \lambda b_i)\\
p_u &\leftarrow p_u + \gamma(e_{ui}*q_i - \lambda p_u)\\
q_i &\leftarrow q_i + \gamma(e_{ui}*p_u - \lambda q_i)
\end{align*}
$$

where $e_{ui} = r_{ui} - \hat r_{ui}$, $\gamma$ is the **learning rat**e and $\lambda$ is the **regularization parameter**.

Now that we understand **SVD** let us train the algorithm on our train set.

In [ ]:
# We'll use the famous SVD algorithm.
algo_svd = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02,random_state=42)

# Train the algorithm on the trainset, and 
algo_svd.fit(trainset)

The **SVD** class takes a few parameters:
+ `n_factors`: The number of factors. Default is 100.
+ `n_epochs`: The number of iteration of the SGD procedure. Default is 20.
+ `lr_all`: The learning rate for all parameters. Default is 0.005.
+ `reg_all`: The regularization term for all parameters. Default is 0.02.   
  

In [ ]:
# Predict ratings for the testset
predictions = algo_svd.test(testset)

# Then compute RMSE
print(f"RMSE: {accuracy.rmse(predictions)}")

**Note**: `.test()` is a method that evaluates the entire test set and returns the predictions as a list of `Prediction` objects. Each object details the `user ID`, `item ID`, `actual rating`, and `estimated rating`. Additionally, the `.predict()` method is used for predicting the rating for a single user-item pair, returning a `Prediction` object that includes the estimated rating among other details.

In [ ]:
for element in predictions:
    print(f"user id:{element.uid}", f"item id:{element.iid}", f"estimated rating:{element.est}", f"real rating:{element.r_ui}")

Let's have a look at the top 10 recommendations for a specific user. Though there is no implementation of this in surprise the documentation provides a function `get_top_n` that returns the top-N recommendations, if we provide the predictions of our model:

In [ ]:
def get_top_n(predictions, n=10):
    """ Return the top-N recommendation for each user from a set of predictions.
    
    Args:
    predictions(list of Prediction objects): The list of predictions, as
        returned by the test method of an algorithm.
    n(int): The number of recommendation to output for each user. Default
        is 10.
    
    
    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of
        size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    
    for user_id, item_id, actual_rating, estimated_rating, _ in predictions:
        top_n[user_id].append((item_id, estimated_rating))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for user_id, estimated_ratings in top_n.items():
        estimated_ratings.sort(key=lambda x: x[1], reverse=True) # sort by rating estimation, descending. x[1] is the estimated rating. 
        top_n[user_id] = estimated_ratings[:n]

    return top_n

What we will get is a list of ten tuples (item_id, estimated_rating). 

In [ ]:
# Getting the top 10 recommendations for each user
top_10 = get_top_n(predictions, n=10)
top_10

In [ ]:
# Print the recommended items for a specific user
user_id = 201   # user id
# 10 best rated items for user id
top_10[user_id]

Let's make list of the top 10 item id's `top_iids`. And use it with the original fishes dataframe to get some characteristics of our recommended fishes. Apparently our user liked especially colorful fishes the most :).

In [ ]:
# The top 10 recommendations for user_id 201 are:
top_items_id_user_id = []
for item_id, estimated_rating in top_10[user_id]:
    print(f"item id: {item_id}, estimated rating: {estimated_rating}")
    top_items_id_user_id.append(item_id)

In [ ]:
top_items_id_user_id

In [ ]:
# Getting the name of the recommended items
recommended_fishes = df.set_index('item_id').loc[top_items_id_user_id][['name','fish_group','visual_effect']].drop_duplicates().copy()
recommended_fishes

## Parameter Grid Search

The SVD algorithm in the surprise library comes with a lot of hyper parameters which influence the performance of the model. To search for optimal hyper parameters in machine learning Grid search is used. It performs an exhaustive search over a prior defined parameter space using cross-validation (hence the CV suffix). That means it will evaluate all of the possible parameter combinations of the search space in order to find and return the best combination.

This task, however, starts to become very time-consuming if there are many hyperparameters and the search space is huge. As you can see for cv= 3 (number of folds that will be evaluated) and for 3 parameters with 2 values, thus 8 combinations, the GridSearchCV runs 24 modelling steps in order to just come up with the best values for the three parameters.

Please feel free to try out different values and beat the default ones from our prediction above ;) and if you still have time improve your result even further!

In [ ]:
param_grid = {
    "n_epochs": [10, 20],       # The number of iteration of the SGD procedure. Default is 20.
    "lr_all": [0.002, 0.005],   # The learning rate for all parameters. Default is 0.005.
    "reg_all": [0.02, 0.04]     # The regularization term for all parameters. Default is 0.02.
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)


In [ ]:

print("best RMSE:")
print(gs.best_score["rmse"])
print("best parameters:")
print(gs.best_params["rmse"])
print("best MAE:")
print(gs.best_score["mae"])
print("best parameters:")
print(gs.best_params["mae"])
print("best estimator:")
print(gs.best_estimator)

In [ ]:
# Get the best model
best_svd = gs.best_estimator["mae"]
# Since GridSearchCV does not train the best_estimator on full data automatically, we need to retrain it:
best_svd.fit(trainset)

print(f"{accuracy.mae(predictions)}")

Keep in mind that getting the best score in recommender systems is not always the best for your business. Very accurate scores can lead to recommendations perceived as boring by the user, because the items are too similar to items they already know. So always keep the user in mind when optimizing your model.

## Predictions for a new user
Let's imagine we have a new user who has not rated any fish yet. This is a common issue called the **cold start problem**. For this user we could use the **most popular** fishes as a recommendation. This is a simple but effective way to start with. We can also ask the user to rate some items and then use the **user-based** or **item-based** collaborative filtering to make recommendations. One could directly use the trained model to make predictions for the new user or retrain the model with the new user's ratings.

In [ ]:
df

We will follow the approach of asking the new user to rate some items and then use the latent factors of the SVD model to make recommendations according to the equation:

$$
\hat r_{ui} = \mu + b_u + b_i + q^T_i p_u
$$

First, we will need to estimate the user latent feature matrix **p** based on the new user's ratings and the item latent feature matrix **q** from the model. We can then use the equation above to predict the ratings for the new user. We will then recommend the top 10 items with the highest predicted ratings.

### Collect Ratings from New User

In [ ]:
new_user_ratings = [
    {"user_id": 500, "item_id": 1, "rating": 10},
    {"user_id": 500, "item_id": 2, "rating": 9},
    {"user_id": 500, "item_id": 3, "rating": 8},
    {"user_id": 500, "item_id": 40, "rating": 7},
    {"user_id": 500, "item_id": 5, "rating": 6},
    {"user_id": 500, "item_id": 6, "rating": 5},
    {"user_id": 500, "item_id": 390, "rating": 4},
    {"user_id": 500, "item_id": 8, "rating": 3},
    {"user_id": 500, "item_id": 9, "rating": 2},
    {"user_id": 500, "item_id": 10, "rating": 1},
]

In [ ]:
# Create a new dataframe with the new user ratings
new_user_ratings_df = pd.DataFrame(new_user_ratings)
new_user_ratings_df

### Estimate the New User's Latent Feature vecture P
We need to estimate the user’s latent feature vector that best explains the new user's provided ratings, using the known item factors from the SVD model.

In [ ]:
# Items that the new user has rated
item_ids = new_user_ratings_df["item_id"].values
item_ids

#### Extract the item factors q for the rated items from the trained model

In [ ]:
item_factors = []
for item_id in item_ids:
    # Convert the raw id to inner id, which is in the same order of the items  as in the model
    item_idx = algo_svd.trainset.to_inner_iid(item_id) 
    #print(item_idx)
    # Get the item factor
    item_factor = algo_svd.qi[item_idx]
    #print(item_factor)
    item_factors.append(item_factor)

# Convert the list of item factors to a numpy array
Q = np.array(item_factors)
# Check the shape of the array
Q.shape

As we can see, `Q` is a numpy array with the shape `(n_items, n_latent_factors)`.

#### Extract the item biases b for the rated items from the trained model

In [ ]:
item_biases = []
for item_id in item_ids:
    # Convert the raw id to inner id, which is in the same order of the items  as in the model
    item_idx = algo_svd.trainset.to_inner_iid(item_id) 
    item_bias = algo_svd.bi[item_idx]
    item_biases.append(item_bias)

b = np.array(item_biases)
b.shape

#### Compute the user bias as the average of all user biases

In [ ]:
# Compute the user bias as the average of all user biases
user_bias = np.mean(algo_svd.bu)
user_bias


#### Compute the user latent feature vector P
To estimate the user latent feature vector, we need to solve the following equation:

$$
\hat r_{ui} = \mu + b_u + b_i + q^T_i p_u
$$

For each rated item, we have the following equation:

$$
r_{ui} = \mu + b_u + b_i + q^T_i p_u
$$

We can rewrite this equation as follows:

$$
r_{ui} - \mu - b_i - b_u = q^T_i p_u
$$

We can then stack these equations for all rated items to form a matrix equation:

$$
R - \mu - B - B_u = Q P_u
$$

where:
- $R$ is the vector of all ratings for the rated items
- $B$ is the vector of all item biases for the rated items
- $B_u$ is the vector of all user biases for the rated items
- $Q$ is the matrix of all item latent feature vectors for the rated items
- $P_u$ is the user latent feature vector we want to estimate
- $\mu$ is the average rating of all items


We can solve this equation for $P_u$ using the following equation:

$$
P_u = (Q^T Q + \lambda I)^{-1} Q^T (R - \mu - B -B_u)

$$

where $\lambda$ is the regularization parameter and $I$ is the identity matrix.



In [ ]:
actual_ratings = new_user_ratings_df['rating'].values
actual_ratings

In [ ]:
# global average rating
mu = algo_svd.trainset.global_mean
mu

In [ ]:
adjusted_ratings = actual_ratings - mu - b - user_bias
adjusted_ratings

In [ ]:
# Compute Q^T Q
QTQ = np.dot(Q.T, Q)
QTQ.shape

In [ ]:
# # Identity matrix of size equal to the number of features
I = np.eye(Q.shape[1])
I.shape

In [ ]:
# Regularized matrix
lambda_reg = 0.1
regularized_matrix = QTQ + lambda_reg * I

In [ ]:
# Inverse of the regularized matrix
inv_regularized_matrix = np.linalg.inv(regularized_matrix)
inv_regularized_matrix.shape

In [ ]:
# Compute final P_u
P_u = inv_regularized_matrix.dot(Q.T).dot(adjusted_ratings)
P_u

In [ ]:
estimated_rating = mu + user_bias + b + np.dot(Q, P_u)
estimated_rating

### Make Recommendations for the New User on Items Not Yet Rated
Now that we have estimated the new user's latent feature vector, we can use it to predict the ratings for the items not yet rated by the user. We can then recommend the top 10 items with the highest predicted ratings.

#### Get all items

In [ ]:

all_items = df['item_id'].unique()
all_items

#### Get the items not rated by the new user

In [ ]:
items_not_rated = np.setdiff1d(all_items, item_ids)
items_not_rated

The numpy function `np.setdiff1d` is a very efficient way to find the difference between two arrays.

#### Extract the item factors for the items not rated by the new user from the trained model

In [ ]:
item_factors_not_rated = []
for item_id in items_not_rated:
    # Convert the raw id to inner id, which is in the same order of the items as in the model
    item_idx = algo_svd.trainset.to_inner_iid(item_id) 
    item_factor = algo_svd.qi[item_idx]
    item_factors_not_rated.append(item_factor)

In [ ]:
# Convert the list to a numpy array
Q_not_rated = np.array(item_factors_not_rated)
Q_not_rated.shape

#### Extract the item biases for the items not rated by the new user from the trained model

In [ ]:
# Extract the item biases for the items not rated by the new user from the trained model
item_biases_not_rated = []
for item_id in items_not_rated:
    # Convert the raw id to inner id, which is in the same order of the items as in the model
    item_idx = algo_svd.trainset.to_inner_iid(item_id)
    item_bias = algo_svd.bi[item_idx]
    item_biases_not_rated.append(item_bias)

In [ ]:
# Convert the list to a numpy array
b_not_rated = np.array(item_biases_not_rated)
b_not_rated

#### Compute the estimated ratings for the items not rated by the new user

In [ ]:

estimated_ratings_not_rated = mu + b_not_rated + user_bias + np.dot(Q_not_rated, P_u)
estimated_ratings_not_rated

#### Get the top 10 items with the highest predicted ratings

In [ ]:
name_of_items_not_rated = df.set_index('item_id').loc[items_not_rated]['name'].drop_duplicates().values
name_of_items_not_rated

In [ ]:
# Create a dataframe with the name of the items and the estimated ratings for the items not rated by the new user
name_estimated_ratings_not_rated_df = pd.DataFrame({
    'name' : name_of_items_not_rated,
    'item_id': items_not_rated,
    'estimated_rating': estimated_ratings_not_rated
})
name_estimated_ratings_not_rated_df


In [ ]:
# Get the name of the top 10 items
top_10_items = name_estimated_ratings_not_rated_df.sort_values(by='estimated_rating', ascending=False).head(10)
top_10_items


## Conclusion
In this notebook, we learned how to use collaborative filtering based on matrix factorization with Singular Value Decomposition (SVD) to make recommendations, by extracting latent features from the user-item interaction matrix. We used the Scikit-Surprise library to train an SVD model on a custom dataset of user ratings for fish items. We learnt then also how to optimize the hyperparameters of the SVD model using GridSearchCV. 

Finally, we made recommendations for a new user who has not rated any items yet. We estimated the new user's latent feature vector using the SVD model and made recommendations for the top 10 items with the highest predicted ratings.

## References
- [Surprise Library](https://surprise.readthedocs.io/en/stable/index.html)
- [SVD](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD)
- [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
- [Singular Value Decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition)
- [Collaborative Filtering](https://en.wikipedia.org/wiki/Collaborative_filtering)
- [Netflix Prize](https://en.wikipedia.org/wiki/Netflix_Prize)
- [Simon Funk](https://sifter.org/simon/journal/20061211.html)
- [Cold Start Problem](https://en.wikipedia.org/wiki/Cold_start_(recommender_systems))
- [Matrix Factorization](https://en.wikipedia.org/wiki/Matrix_factorization_(recommender_systems))
